[Reference](https://python.plainenglish.io/continue-developing-api-with-fastapi-and-postgresql-beyond-the-get-method-937a9010583f)

- POST — to write new data
- DELETE — removes your data
- PUT — updates existing information

In [3]:
# schemas.py
from pydantic import BaseModel
from typing import Optional

class StockBase(BaseModel):
    symbol: str
    stockname: str
    lastsale: str
    country: str
    ipoyear: Optional[int] = None
    volume: int

class StockCreate(StockBase):
    pass

class Stock(StockBase):

    class Config:
        orm_mode = True

class UserBase(BaseModel):
    user_id: str

class User(UserBase):
    first_name: str
    last_name: str
    last_sale_amount: int

    class Config:
        orm_mode = True

class UserCreate(User):
    pass

class UserUpdateSaleAmount(UserBase):
    last_sale_amount: int

    class Config:
        orm_mode = True

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [6]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

from config import settings

engine = create_engine(
    settings.sqlalchemy_string, connect_args={'sslmode':'require'}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

In [8]:
from sqlalchemy import Column, Integer, String, Float, BigInteger

class Stock(Base):
    __tablename__ = "nasdaq_stocks"

    symbol = Column(String, primary_key=True)
    stockname = Column(String)
    lastsale = Column(String)
    netchange = Column(Float)
    percentchange = Column(String)
    marketcap = Column(BigInteger)
    country = Column(String, nullable=True)
    ipoyear = Column(Integer, nullable=True)
    volume = Column(Integer)
    sector = Column(String, nullable=True)
    industry = Column(String, nullable=True)

class User(Base):
    __tablename__ = "users"

    user_id = Column(String, primary_key=True)
    first_name = Column(String)
    last_name = Column(String)
    last_sale_amount = Column(Integer)

In [9]:
from sqlalchemy.orm import Session

import models, schemas

def get_stock(db: Session, symbol: str):
    return db.query(models.Stock).filter(models.Stock.symbol == symbol).first()

def post_stock(db: Session):
    pass

def get_user(db: Session, user_id: str):
    return db.query(models.User).filter(models.User.user_id == user_id).first()

def post_user(db: Session, user_data: schemas.UserCreate):
    db.begin()
    db.add(models.User(user_id=user_data.user_id,
                       first_name=user_data.first_name,
                       last_name=user_data.last_name,
                       last_sale_amount=user_data.last_sale_amount))
    db.commit()

def put_user_sale(db: Session, user_data: schemas.UserUpdateSaleAmount):
    db.query(models.User).filter(models.User.user_id == user_data.user_id).update(
        {
            "last_sale_amount": user_data.last_sale_amount,
        }
    )
    db.commit()

def delete_user(db: Session, user_id: str):
    db.query(models.User).filter(models.User.user_id == user_id).delete()
    db.commit()

In [10]:
from fastapi import FastAPI, HTTPException, Query, Depends
from sqlalchemy.orm import Session

from database import SessionLocal, engine

models.Base.metadata.create_all(bind=engine)

app = FastAPI(
    title="NASDAQ stocks",
    description="Start using FastAPI in development",
    version="0.1"
)

# Dependency
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


@app.get('/stock/{symbol}', response_model=schemas.Stock, status_code=200)
def get_stock(symbol: str, db: Session = Depends(get_db)) -> models.Stock:
    db_stock = crud.get_stock(db, symbol=symbol)
    if db_stock is None:
        raise HTTPException(
            status_code=404, detail=f"No stock {symbol} found."
        )

    return db_stock

@app.get('/user/{user_id}', response_model=schemas.User, status_code=200)
def get_user(user_id: str, db: Session = Depends(get_db)) -> models.User:
    db_user = crud.get_user(db, user_id=user_id)
    if db_user is None:
        raise HTTPException(
            status_code=404, detail=f"No user {user_id} found."
        )

    return db_user

@app.post("/create_user/")
async def post_user(user: schemas.UserCreate, db: Session = Depends(get_db)):
    try:
        crud.post_user(db, user)
    except Exception as e:
        raise e

    return {"response": f"User {user.user_id} was created."}

@app.put("/update_user_last_sale/")
async def update_user_last_sale(user: schemas.UserUpdateSaleAmount, db: Session = Depends(get_db)):
    try:
        crud.put_user_sale(db, user)
    except Exception as e:
        raise e

    return {"response": f"User {user.user_id} data was updated."}

@app.delete("/delete_user/{user_id}")
def delete_user(user_id: str, db: Session = Depends(get_db)):
    try:
        crud.delete_user(db, user_id)
    except Exception as e:
        raise e

    return {"response": f"User {user_id} is removed."}

In [11]:
import requests
import random
import names

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

for i in range(1, 100):
    json_data = {
        'user_id': str(i),
        'first_name': names.get_first_name(),
        'last_name': names.get_last_name(),
        'last_sale_amount': random.randint(1, 200000),
    }

    response = requests.post('http://127.0.0.1:8000/user/', headers=headers, json=json_data)
    print(response.json())

# Implementing PUT method


In [12]:
import requests
import random

print(requests.get('http://127.0.0.1:8000/user/2').json())

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

json_data = {
  "user_id": "2",
  "last_sale_amount": random.randint(0, 200000)
}

response = requests.put('http://127.0.0.1:8000/update_user_last_sale/', headers=headers, json=json_data)
print(response.json())

print(requests.get('http://127.0.0.1:8000/user/2').json())

# Implementing DELETE method

In [13]:
import requests

delete_user = "5"
print(requests.delete(f"http://127.0.0.1:8000/delete_user/{delete_user}").json())
print(requests.get(f"http://127.0.0.1:8000/user/{delete_user}").json())